In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import requests
import csv

import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [20]:
# Parameters section

alpha_vantage_api_key = 'YI2V50P8VRQ3HFKM' # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# # Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [
    'SYK'
]

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]


ticker_dict = {}

In [48]:
# Daily quote section
for symbol in ticker_symbols:

    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':

            # selected_cols = [
            #     '4. close'
            # ]

            df_stock_base = pd.DataFrame(value).transpose() # tranpose the dataframe and sub select selected cols

            # Rename columns
            df_stock_base.rename(
                columns={
                    '1. open': 'open'
                    ,'2. high': 'high'
                    ,'3. low': 'low'
                    ,'4. close': 'close'
                    ,'5. volume': 'volume'
                    }
                ,inplace=True
                )
            
            for col in df_stock_base.columns:
                if col == 'volume':
                    df_stock_base[col] = df_stock_base[col].astype(str).apply(lambda x: int(x))
                else:
                    df_stock_base[col] = df_stock_base[col].astype(str).apply(lambda x: float(x))
                    df_stock_base[col] = df_stock_base[col].round(2)


            df_stock_base['symbol'] = symbol
            # convert string index to datetime index
            df_stock_base.index = pd.to_datetime(df_stock_base.index)


            # 锤子线
            
            # 分为当日是阴线， 阳线
            # 1. 出现在明显的下行趋势中
            # 过去一段period里的斜率是负数
            # max() high 与 min() high 之间的差值有超过10% 或者 15%，可以调参


            # 2. 下影线是实体部分的两倍以上



            # 3. 成交量不能是缩量， 至少要和平均成交量持平， 越大越好
            # 当日成交量>=，过去一段时间成交量的均值, 或者前75%


In [63]:
# 锤子线

# 分为当日是阴线， 阳线
# 1. 出现在明显的下行趋势中
# 过去一段period里的斜率是负数


# 2. 下影线是实体部分的两倍以上

# 3. 成交量不能是缩量， 至少要和平均成交量持平， 越大越好
# 当日成交量>=，过去一段时间成交量的均值, 或者前75%

def K_hammer(window):
    # in pervious window days
    window = window
    # 阴线
    for index, row in df_stock_base.iterrows():
        if row['open'] > row['close']:

            print(index, "open > close")
            # print(index-timedelta(1))
            print(df_stock_base[index])
            # print(df_stock_base['high'].max())
            # if (max(row['high']) - min(row['high'])) / max(row['high']) >= 0.15 \
            #       and abs(row['close'] - row['low']) >= 2 * abs(row['open'] - row['close']) \
            #       and min(row['high'].index) > max(row['high'].index) \
            #       and  


        # 阳线
        if row['open'] < row['close']:
            print(index, "open < close")
        # print(df_stock_base[index])

    return

In [64]:
K_hammer(7)

2024-05-17 00:00:00 open < close
2024-05-16 00:00:00 open < close
2024-05-15 00:00:00 open < close
2024-05-14 00:00:00 open < close
2024-05-13 00:00:00 open > close


KeyError: Timestamp('2024-05-13 00:00:00')

In [27]:
df_stock_base

,open,high,low,close,volume,symbol
2024-05-17,334.60,334.98,330.46,334.68,937468,SYK
2024-05-16,332.00,335.00,331.35,333.98,1168494,SYK
2024-05-15,325.91,332.22,324.30,331.99,1989178,SYK
2024-05-14,324.09,326.02,322.50,324.30,1712601,SYK
2024-05-13,331.32,331.32,323.51,324.56,1609587,SYK
2024-05-10,331.80,332.82,330.15,330.58,1307926,SYK
2024-05-09,330.41,332.75,329.29,330.86,980439,SYK
2024-05-08,331.94,335.40,327.60,329.28,1214050,SYK
2024-05-07,327.93,332.39,326.63,332.14,1331236,SYK
2024-05-06,330.30,331.00,324.59,325.50,1722244,SYK


In [25]:
def stop_loss_calculator(buy_qty, buy_price, stop_loss_factor):

    '''
    This is a calculator to return the exist stop loss price of a trade

    input parameters:
        buy_qty: the total quantity of security bought, INTEGER
        buy_price: the unit share price of the security bought,buy_qty, buy_price, DECIMAL
        stop_loss_factor: the percentage trader is willing to lose at the exist position, DECIMAL
    '''

    total_deployed_cap = buy_qty * buy_price
    stop_loss_price = round(buy_price - ((total_deployed_cap * stop_loss_factor) / buy_qty), 2)
    
    return stop_loss_price



def stop_profit_calculator(buy_qty, buy_price, stop_profit_factor):

    '''
    This is a calculator to return the exist stop profit price of a trade

    input parameters:
        buy_qty: the total quantity of security bought, INTEGER
        buy_price: the unit share price of the security bought, DECIMAL
        stop_profit_factor: the percentage trader is willing to lose at the exist position, DECIMAL
    '''

    total_deployed_cap = buy_qty * buy_price
    stop_profit_price = round((total_deployed_cap * (1+stop_profit_factor)) / buy_qty, 2)

    return stop_profit_price

In [3]:
stop_loss_calculator(20, 150.02, 0.01)

148.52

In [4]:
stop_profit_calculator(20, 150.02, 0.10)

165.02

In [68]:
pip install TA-Lib


     ---------------------------------------- 0.0/357.1 kB ? eta -:--:--
     ---------------- --------------------- 153.6/357.1 kB 3.1 MB/s eta 0:00:01
     -------------------------------------- 357.1/357.1 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build TA-Lib


  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      <string>:77: UserWarning: Cannot find ta-lib library, installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\talib
      copying talib\abstract.py -> build\lib.win-amd64-cpython-39\talib
      copying talib\deprecated.py -> build\lib.win-amd64-cpython-39\talib
      copying talib\stream.py -> build\lib.win-amd64-cpython-39\talib
      copying talib\__init__.py -> build\lib.win-amd64-cpython-39\talib
      running build_ext
      building 'talib._ta_lib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates 

In [66]:
pip install mpl_finance